In [ ]:
import os

import pandas as pd

import numpy as np

import random
import gzip

import pickle

from tqdm import tqdm

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, SelectFromModel

from imblearn.under_sampling import RepeatedEditedNearestNeighbours

### Find cell markers

In [ ]:
meta = pd.read_csv("../../02_rds/meta_after_singleR.csv", index_col=0)
meta = meta.loc[(meta['Batch'] != 3) & (meta["cell_name"].notna()), :]

print(meta.shape)

In [ ]:
def choose_cells(meta, seed = 42, n=1000):
    u"""
    
    """
    choosed_cells = []

    random.seed(seed)
    for i in meta["cell_name"].unique():
        cells = meta.loc[meta["cell_name"] == i, "Cells"]

        if len(cells) <= n:
            choosed_cells += list(cells)
        else:
            choosed_cells += random.choices(list(cells), k=n)
    
    return choosed_cells

choosed_cells = choose_cells(meta)

In [ ]:


def read_scale_data(path, cells=None):
    
    header = {}
    res = {}
    with gzip.open(path, "rt") as r:
        for line in tqdm(r):
            lines = line.strip().replace("\"", "").split(",")
            if len(header) == 0:
                for i, j in enumerate(lines):
                    header[j] = i + 1
                
                if not cells:
                    cells = list(header.keys())
            else:
                temp = {k: float(lines[header[k]]) for k in cells}
                res[lines[0]] = temp
    return pd.DataFrame(res)

expr = read_scale_data("scale_data.csv.gz", choosed_cells)
    
with open("./choosed_all_cells_scale_expr.pickle", "wb+") as w:
    pickle.dump(expr, w)
    
expr = expr.transpose()

In [ ]:
with open("./choosed_all_cells_scale_expr.pickle", "rb") as r:
    expr = pickle.load(r)

Read features

In [16]:
"""
write.csv(tf, here("12_TF/Teichmann_TFlist.csv"))

write.csv(sf, here("12_TF/Han_Molecular_Cell.csv"))

write.csv(rbp, here("12_TF/RBP.csv"))
"""

tf = pd.read_csv("../12_TF/Teichmann_TFlist.csv", index_col=0)
sf = pd.read_csv("../12_TF/Han_Molecular_Cell.csv", index_col=0)
rbp = pd.read_csv("../12_TF/RBP.csv", index_col=0)


In [11]:
def find_best_features_by_cv(
    data, 
    cluster,
    init_features=None, 
    n_iter=10, 
    random_state=0, 
    n_estimitors=100, 
    max_depth=2, 
    n_jobs=10,
    cv=10,
    methods="RF"
):
    u"""
    Find best features by crossvalidation
    :param data: 
    """
    clf = None
    if methods == "RF":
        clf = RandomForestClassifier(n_estimators=n_estimitors, max_depth=max_depth, random_state=random_state, n_jobs=n_jobs)
    elif methods == "SVM":
        clf = SVC(
            C=1.0, 
            kernel=i, 
            degree=3, 
            gamma='auto_deprecated', 
            coef0=0.0, 
            shrinking=True, 
            probability=False, 
            tol=0.001, 
            cache_size=200, 
            class_weight=None, 
            verbose=False, 
            max_iter=-1, 
            decision_function_shape='ovr', 
            random_state=random_state
        )
    elif methods == "KNN":
        clf = KNeighborsClassifier(n_neighbors=30)
    elif methods == "ANN":
        clf = MLPClassifier()
    
    if init_features is None:
        init_features = data.index 
        
    init_features = list(set(init_features) & set(data.index))
            
    res = []
    for i in range(n_iter):
        mat = data.loc[init_features, :].transpose()
        cv_results = cross_validate(clf, X=mat, y=cluster, cv=cv, n_jobs=n_jobs, return_estimator=True)
    
        res.append(
            {
                "iter": i,
                "test_score": cv_results["test_score"],
                "features": init_features
            }
        )
        
        importance = [] 
        for idx, estimator in enumerate(cv_results["estimator"]):
            importance.append(
                pd.DataFrame(
                    estimator.feature_importances_,
                    index=init_features,
                )
            )


        importance = pd.concat(importance, axis=1)
        
        importance = importance > 0
        importance = importance.sum(axis=1)
        
        init_features = importance[importance > (cv / 2)].index
        
    return res

In [42]:
def find_best_features_by_select_from_model(
    data, 
    cluster,
    init_features=None, 
    n_iter=10, 
    random_state=0, 
    n_estimitors=100, 
    max_depth=2, 
    n_jobs=10,
    cv=10,
    methods="RF"
):
    u"""
    Find best features by crossvalidation
    :param data: 
    """
    clf = None
    if methods == "RF":
        clf = RandomForestClassifier(n_estimators=n_estimitors, max_depth=max_depth, random_state=random_state, n_jobs=n_jobs)
    elif methods == "SVM":
        clf = SVC(
            C=1.0, 
            kernel=i, 
            degree=3, 
            gamma='auto_deprecated', 
            coef0=0.0, 
            shrinking=True, 
            probability=False, 
            tol=0.001, 
            cache_size=200, 
            class_weight=None, 
            verbose=False, 
            max_iter=-1, 
            decision_function_shape='ovr', 
            random_state=random_state
        )
    elif methods == "KNN":
        clf = KNeighborsClassifier(n_neighbors=30)
    elif methods == "ANN":
        clf = MLPClassifier()
    
    if init_features is None:
        init_features = data.index 
        
    init_features = list(set(init_features) & set(data.index))
            
    res = []
    for i in range(n_iter):
        mat = data.loc[init_features, :].transpose()

        X_train, X_test, y_train, y_test = train_test_split(mat, cluster, test_size=0.33, random_state=random_state)

        sel = SelectFromModel(clf)
        sel.fit(X_train, y_train)

        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        res.append(
            {
                "iter": i,
                "test_score": accuracy_score(y_test, pred, normalize=True, sample_weight=None),
                "features": init_features
            }
        )
        
        init_features = [init_features[x] for x in sel.get_support()]
        
    return res

## All cells

In [ ]:
res = find_best_features_by_cv(expr, meta.loc[expr.columns, "cell_name"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

In [ ]:
res = find_best_features_by_select_from_model(expr, meta.loc[expr.columns, "cell_name"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

In [ ]:
raw = read_scale_data("LungCancer10x/02_rds/raw_data.csv.gz", choosed_cells)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X_new = SelectKBest(chi2, k=2).fit_transform(raw, meta.loc[expr.columns, "cell_name"])

In [ ]:
from feature_selector import FeatureSelector

fs = FeatureSelector(data = expr, labels = list(meta.loc[expr.index, "cell_name"]))



fs.identify_zero_importance(task = 'classification', eval_metric = 'auc', n_iterations = 10, early_stopping = False)



### Basal

In [19]:
basal = pd.read_csv("../03_each_cells/LUSC/Basal/scanpy/scale.csv.gz", index_col=0)

In [20]:
res = find_best_features_by_cv(basal, list(meta.loc[basal.columns, "Stage"]), init_features=tf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.7157553909543188
0.7307867134462194
0.747131058925462
0.742642778711588
0.7484510071685191
0.7471358791232889
0.7563575661221767
0.7563575661221767
0.7563575661221767
0.7563575661221767


In [21]:
res = find_best_features_by_cv(basal, meta.loc[basal.columns, "Stage"], init_features=sf["SYMBOL"])
for i in res:
    print(np.mean(i["test_score"])) # i["test_score"],

0.7378961864595421
0.793539423447179
0.834130529198362
0.8314975139980854
0.8375654505368008
0.8291249158938687
0.8378126601808897
0.8412503845990227
0.8433646816575369
0.8431015274365989


In [22]:
res = find_best_features_by_cv(basal, meta.loc[basal.columns, "Stage"], init_features=rbp["SYMBOL"])
for i in res:
    print(np.mean(i["test_score"])) # i["test_score"],

0.8586487210206905
0.8723531370364105
0.8747222340391267
0.8612301705470633
0.8702346188696509
0.8734029618329371
0.8749888783690956
0.8749847122231973
0.8749847122231973
0.8749847122231973


### ATII

In [23]:
atii = pd.read_csv("../03_each_cells/total/Alveolar_II/scanpy/scale.csv.gz", index_col=0)

In [25]:
res = find_best_features_by_cv(atii, list(meta.loc[atii.columns, "Disease"]), init_features=tf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.659184847020733
0.6624508692750506
0.6731407241191077
0.6761775928431836
0.6819190757644058
0.670665265732361
0.6755033102868742
0.6776394136011523
0.6776394136011523
0.6776394136011523


In [27]:
res = find_best_features_by_cv(atii, list(meta.loc[atii.columns, "Disease"]), init_features=sf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.6578342551421146
0.6610981287818902
0.6692026974545231
0.6647003415472568
0.6775237650931153
0.6780903699016227
0.675274543615288
0.6727966861100564
0.6729144909155892
0.6774067228973435


In [46]:
res = find_best_features_by_cv(atii, list(meta.loc[atii.columns, "Disease"]), init_features=rbp["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.6676286520687273
0.71442302085965
0.72938684182337
0.733213397077084
0.7332130176378648
0.7309643034093188
0.7354570457754829
0.7341178388752109
0.7352418126871663
0.7368147237030804


select best features by select from model

In [43]:
res = find_best_features_by_select_from_model(atii, list(meta.loc[atii.columns, "Disease"]), init_features=tf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"]))

program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an err

In [44]:
res = find_best_features_by_select_from_model(atii, list(meta.loc[atii.columns, "Disease"]), init_features=sf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"]))

program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an err

In [45]:
res = find_best_features_by_select_from_model(atii, list(meta.loc[atii.columns, "Disease"]), init_features=rbp["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"]))

program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
program/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: In future, it will be an err

#### ATII luad

In [72]:
atii_luad = pd.read_csv("../03_each_cells/LUAD/Alveolar_II/scanpy/scale.csv.gz", index_col=0)

target_cells = meta.loc[meta["Disease"] == "LUAD", :]
atii_luad = atii_luad.loc[:, set(atii_luad.columns) & set(target_cells.index)]


In [73]:
res = find_best_features_by_cv(atii_luad, list(meta.loc[atii_luad.columns, "Stage"]), init_features=tf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.6200750469043153
0.6864915572232646
0.6839274546591619
0.6888055034396497
0.6938086303939963
0.6758599124452783
0.6862414008755472
0.6886804252657912
0.6835522201375861
0.6835522201375861


In [74]:
res = find_best_features_by_cv(atii_luad, list(meta.loc[atii_luad.columns, "Stage"]), init_features=sf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.5661038148843026
0.6271419637273297
0.6424015009380863
0.627016885553471
0.6480300187617262
0.6325203252032521
0.6294559099437148
0.6348342714196373
0.6348342714196373
0.6348342714196373


In [75]:
res = find_best_features_by_cv(atii_luad, list(meta.loc[atii_luad.columns, "Stage"]), init_features=rbp["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.6276422764227643
0.6426516572858038
0.6580362726704191
0.6608505315822389
0.6580362726704191
0.6709818636647905
0.6732958098811758
0.6706066291432146
0.6706066291432146
0.6706066291432146


### CD4

In [54]:
cd4 = pd.read_csv("../03_each_cells/total/CD4/scanpy/scale.csv.gz", index_col=0)

In [55]:
res = find_best_features_by_cv(cd4, list(meta.loc[cd4.columns, "Disease"]), init_features=tf["SYMBOL"])

for i in res:
    print(np.mean(i["test_score"])) # i["test_score"], 

0.4282931055566393
0.42781318450216743
0.42946054701107894
0.4289112843697772
0.42774403153597235
0.4285677835633134
0.4285685378502654
0.4285685378502654
0.4285685378502654
0.4285685378502654


In [69]:
cd4_luad = meta.loc[(meta["Disease"] == "LUAD") & (meta["cell_name"] == "CD4") & (meta["Batch"] != 3), ]
# cd4_luad = cd4.loc[:, cd4_luad]
# res = find_best_features_by_cv(cd4_luad.loc[:, ], list(meta.loc[cd4_luad.columns, "Disease"]), init_features=tf["SYMBOL"])

# for i in res:
#     print(np.mean(i["test_score"])) # i["test_score"], 

In [70]:
expr = cd4.loc[:, cd4_luad.index]

expr.head()

,2018jz1_AACCGCGCAGTGGGAT-1,2018jz1_AAGCCGCAGGCATGTG-1,2018jz1_AAGCCGCTCTCGCTTG-1,2018jz1_ACACCCTTCAAACGGG-1,2018jz1_ACGGGCTCATCGGTTA-1,2018jz1_ACTGATGGTCGCATAT-1,2018jz1_AGCTTGATCCGGCACA-1,2018jz1_ATTACTCCAGGAATGC-1,2018jz1_CAAGATCAGATGCCTT-1,2018jz1_CCATGTCAGATCGATA-1,...,AK657_TACCTATGTAAGAGAG-1,AK657_TACTTGTGTGACCAAG-1,AK657_TCAGCAACAATAGCAA-1,AK657_TGCGCAGAGAGTCGGT-1,AK657_TGCGGGTCATGAGCGA-1,AK657_TTAGGACAGCTTTGGT-1,AK657_TTAGGCACAACTTGAC-1,AK657_TTCGGTCGTGTGCGTC-1,AK657_TTGCGTCAGATACACA-1,AK657_TTTCCTCGTTCCCTTG-1
A1BG,NaN,NaN,NaN,-0.079085,NaN,-0.085342,NaN,-0.085257,NaN,-0.086009,...,-0.099788,NaN,-0.062124,-0.103510,NaN,NaN,-0.063538,-0.066060,NaN,-0.061271
A1BG-AS1,NaN,NaN,NaN,-0.098340,NaN,-0.097162,NaN,-0.094569,NaN,-0.093521,...,-0.079549,NaN,-0.108655,-0.070391,NaN,NaN,-0.105155,-0.106234,NaN,-0.104626
A2M,NaN,NaN,NaN,-0.023550,NaN,-0.027839,NaN,-0.040404,NaN,-0.045303,...,-0.109799,NaN,0.022535,-0.153271,NaN,NaN,0.005918,0.011707,NaN,0.002850
A2M-AS1,NaN,NaN,NaN,-0.075331,NaN,-0.067152,NaN,-0.082471,NaN,-0.086770,...,-0.135083,NaN,-0.055987,-0.179516,NaN,NaN,-0.072991,-0.060636,NaN,-0.081522
A4GALT,NaN,NaN,NaN,-0.022631,NaN,-0.027135,NaN,-0.025305,NaN,-0.025232,...,-0.027436,NaN,-0.015251,-0.024382,NaN,NaN,-0.014076,-0.016945,NaN,-0.011797
